# Working with Together AI in Pixeltable

### Prerequisites

- A Together AI account with an API key (https://api.together.ai/settings/api-keys)

### Important notes

- Together.ai usage may incur costs based on your Together.ai plan.
- Be mindful of sensitive data and consider security measures when integrating with external services.

First you'll need to install required libraries and enter your Together API key.

In [ ]:
%pip install -qU pixeltable together

In [ ]:
import os
import getpass

if 'TOGETHER_API_KEY' not in os.environ:
    os.environ['TOGETHER_API_KEY'] = getpass.getpass('Together API Key: ')

Now let's create a Pixeltable directory to hold the tables for our demo.

In [ ]:
import pixeltable as pxt

# Remove the 'together_demo' directory and its contents, if it exists
pxt.drop_dir('together_demo', force=True)
pxt.create_dir('together_demo')

## Chat completions

Create a Table: In Pixeltable, create a table with columns to represent your input data and the columns where you want to store the results from OpenAI.

In [ ]:
from pixeltable.functions import together

chat_t = pxt.create_table('together_demo.chat', {'input': pxt.String})

messages = [{'role': 'user', 'content': chat_t.input}]

chat_t.add_computed_column(output=together.chat_completions(
    messages=messages,
    model='meta-llama/Llama-3.3-70B-Instruct-Turbo',
    model_kwargs={
        # Optional dict with parameters for the Together API
        'max_tokens': 300,
        'stop': ['\n'],
        'temperature': 0.7,
        'top_p': 0.9,
    }
))
chat_t.add_computed_column(response=chat_t.output.choices[0].message.content)

In [ ]:
# Start a conversation
chat_t.insert([
    {'input': 'How many species of felids have been classified?'},
    {'input': 'Can you make me a coffee?'}
])
chat_t.select(chat_t.input, chat_t.response).head()

## Embeddings

In [ ]:
emb_t = pxt.create_table('together_demo.embeddings', {'input': pxt.String})
emb_t.add_computed_column(embedding=together.embeddings(
    input=emb_t.input,
    model='BAAI/bge-base-en-v1.5'
))

In [ ]:
emb_t.insert([{'input': 'Together AI provides a variety of embeddings models.'}])

In [ ]:
emb_t.head()

## Image generations

In [ ]:
image_t = pxt.create_table('together_demo.images', {'input': pxt.String})
image_t.add_computed_column(img=together.image_generations(
    image_t.input,
    model='black-forest-labs/FLUX.1-schnell',
    model_kwargs={'steps': 5}
))

In [ ]:
image_t.insert([
    {'input': 'A friendly dinosaur playing tennis in a cornfield'}
])

In [ ]:
image_t

In [ ]:
image_t.head()

### Learn more

To learn more about advanced techniques like RAG operations in Pixeltable, check out the [RAG Operations in Pixeltable](https://docs.pixeltable.com/howto/use-cases/rag-operations) tutorial.

If you have any questions, don't hesitate to reach out.